# Distributed Tracing Template

 Illustrate the configuration for allowing distributed tracing using Jaeger.
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Install Jaeger

We will use the Jaeger All-in-1 resource found at the [Jaeger Kubernetes repo](https://github.com/jaegertracing/jaeger-kubernetes).

In [ ]:
!kubectl create -f https://raw.githubusercontent.com/jaegertracing/jaeger-kubernetes/master/all-in-one/jaeger-all-in-one-template.yml -n seldon

### Start Jaeger UI

```
minikube service jaeger-query -n seldon
```

## Install Seldon

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true --namespace seldon
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon

## Create Jaeger ConfigMap

In [ ]:
!pygmentize tracing-configmap.yaml

In [1]:
!kubectl apply -f tracing-configmap.yaml -n seldon

configmap/tracing-config created


## Run Example REST Deployment

In [14]:
!pygmentize deployment_rest.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "tracing-example",
	"namespace": "seldon"	
    },
    "spec": {
        "name": "tracing-example",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "name": "model1",				
                                "image": "seldonio/mock_classifier_rest:1.1",
				"env": [
				    {
					"name": "TRACING",
					"value": "1"
				    },
				    {
					"name": "JAEGER_CONFIG_PATH",
					"value": "/etc/tracing/config/tracing.yml"
				    }
				],
				"volumeMounts": [
				    {
					"mountPath": "/etc/tracing/config",
					"name": "tracing-config"
				    }
				]
                            }
		

In [15]:
!kubectl create -f deployment_rest.json

seldondeployment.machinelearning.seldon.io/tracing-example created


In [ ]:
!kubectl get sdep tracing-example -o jsonpath='{.status}' 

In [20]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['feat1', 'feat2', 'feat3'], 'ndarray': [[-1.7476463302482368, 1.0403015758607754, -0.5845863387322653]]}}
Getting token from http://192.168.39.75:31888/oauth/token
{"access_token":"5dfe7536-83ab-4882-90c2-4fe357fe5964","token_type":"bearer","expires_in":42525,"scope":"read write"}
RECEIVED RESPONSE:
{'meta': {'puid': '9imi6g2q72iujm0qqffhd9bpj9', 'tags': {}, 'routing': {}, 'requestPath': {'model1': 'seldonio/mock_classifier_rest:1.1'}, 'metrics': []}, 'data': {'names': ['proba'], 'ndarray': [[0.033983350227398276]]}}



Check the Jaeger UI. You should be able to find traces like below:

![rest](jaeger-ui-rest-example.png)

In [21]:
!kubectl delete -f deployment_rest.json

seldondeployment.machinelearning.seldon.io "tracing-example" deleted


## Run Example GRPC Deployment

In [ ]:
!kubectl create -f deployment_grpc.json

In [ ]:
!kubectl get sdep tracing-example -o jsonpath='{.status}' 

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc \
    --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

Check the Jaeger UI. You should be able to find traces like below:


![grpc](jaeger-ui-grpc-example.png)

In [ ]:
!kubectl delete -f deployment_grpc.json